In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 

from transformers import BertForSequenceClassification, BertTokenizer, BertForMaskedLM

from simpletransformers.language_modeling import LanguageModelingModel

from sklearn.metrics.pairwise import cosine_similarity, paired_euclidean_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import normalize, StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import itertools
from collections import Counter

from tqdm import tqdm
import torch

import networkx as nx

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from functools import partial

import pickle

from collections import deque

from torchvision import models
import torch.nn.functional as F
from torch import nn

from sklearn.utils import resample

import fasttext
import sister



from typing import List
from pathlib import Path
from joblib import dump, load

import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/home/roshansk/YelpAnalysis/')
from utils import *

import spacy

# ft = fasttext.load_model('/home/roshansk/YelpAnalysis/RandomForest/cc.en.300.bin')

# embedder = sister.MeanEmbedding(lang="en")





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
nlp = spacy.load('/data2/link10/models/fasttext/en_fasttext_crawl')
# nlp = spacy.load('/data2/link10/models/fasttext/en_fasttext_crawl_subword/')

sys.path.insert(0,'/home/roshansk/YelpAnalysis/Datasets/')
from createDataset import *

dataFolder = '/home/roshansk/YelpAnalysis/Datasets/'

### Helper Functions

In [3]:
def trainFFN(trainData, testData, num_epochs = 5, batchSize = 5, device='cuda:0'):
    
        
    NNnet = NNNet()    
    NNnet.to(device)

    classLoss = nn.CrossEntropyLoss()
    reconLoss = nn.MSELoss()


    optimizer_NN = torch.optim.Adam(filter(lambda p: p.requires_grad, NNnet.parameters()), lr=0.0001)

    

    trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batchSize, 
                                              shuffle=True, num_workers=1)


    testLoader = torch.utils.data.DataLoader(testDataset, batch_size=batchSize, 
                                              shuffle=False, num_workers=1)
    
    
    for i in range(num_epochs):
        train_epoch_NN(NNnet, trainLoader, testLoader, optimizer_NN)
        
        
    return NNnet
    
    
def generateLexicon(NNnet, trainDf, device = 'cuda:0'):
    
    wordPred = getWordPred(NNnet, trainDf, nlp, device)
    
    return wordPred

### Load Dataset

In [18]:
trainDf, devDf, testDf = getData(dataFolder, 'nrc_joy')

In [19]:
trainData = generateFastTextData_Spacy(trainDf, nlp, textVariable = 'text')

testData = generateFastTextData_Spacy(testDf, nlp, textVariable = 'text')

trainDataset = Dataset(trainDf, trainData)
testDataset = Dataset(testDf, testData)

### Model Training

In [20]:
NNnet = trainFFN(trainData, testData, num_epochs = 3)

0it [00:00, ?it/s]/home/roshansk/YelpAnalysis/utils.py:530: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.

421it [00:01, 321.08it/s]


Acc : 0.7448931116389549 F1 : 0.7060755336617406
*************************


421it [00:01, 234.20it/s]

Acc : 0.7700712589073634 F1 : 0.7070217917675544
*************************



421it [00:01, 214.50it/s]

Acc : 0.7605700712589074 F1 : 0.6589986468200271
*************************


In [21]:
lexicon = generateLexicon(NNnet,trainDf, device ='cuda:0')

In [22]:
# vector = nlp('bookmark').vector
# NNnet(torch.from_numpy(vector.reshape(1,-1)).to('cuda:0'))

In [23]:
lexicon.rename({'NNprob':'score'},axis = 1, inplace = True)
lexiconWords, lexiconMap = getLexicon(0, df = lexicon)



  ,  , 0.866 , 0.913 , 0.973 , 0.923


### Testing

In [12]:
def testFFN_direct( NNnet, dataset, lexiconWords, lexiconMap, nlp, train = False):
    
    if dataset != 'dialog':
        trainDf, devDf, testDf = getData(dataFolder, dataset)
    else:
        testDf = getExternalData(dataFolder, 'dialog')

        
    if train:
        testData = generateFastTextData_Spacy(trainDf, nlp, textVariable = 'text')
        testDataset = Dataset(trainDf, testData)
        testDf = trainDf
    else:
        testData = generateFastTextData_Spacy(testDf, nlp, textVariable = 'text')
        testDataset = Dataset(testDf, testData)
        
    testLoader = torch.utils.data.DataLoader(testDataset, batch_size=5, 
                                              shuffle=False, num_workers=1)
    
    
    modelAcc, modelF1 = testModel_NN(NNnet, testLoader)
    
#     print(f"Acc : {acc} F1 : {f1}" )
    
    
    lexAcc, lexF1 = evaluateLexicon(testDf, lexiconWords, lexiconMap, nlp, returnScores = True)
    
    print(f"{dataset} , {modelAcc} , {modelF1} , {lexAcc} , {lexF1}")
    

In [13]:
testFFN_direct(NNnet,'dialog',lexiconWords, lexiconMap, nlp)

2708it [00:08, 306.25it/s]


dialog , 0.7348205052444969 , 0.8156137647663072 , 0.828 , 0.901


In [10]:
testFFN_direct(NNnet,'dialog',lexiconWords, lexiconMap, nlp)

In [24]:
dataList = ['dialog','amazon_finefood_subset','amazon_toys_subset','yelp_subset','empathy','nrc_joy']
for data in dataList:
    testFFN_direct(NNnet,data,lexiconWords, lexiconMap, nlp)

2708it [00:07, 363.19it/s]


dialog , 0.4027182744866302 , 0.4372216035634744 , 0.815 , 0.898


0it [00:00, ?it/s]/home/roshansk/YelpAnalysis/utils.py:530: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.

200it [00:00, 222.91it/s]


amazon_finefood_subset , 0.308 , 0.32289628180039137 , 0.85 , 0.919


0it [00:00, ?it/s]/home/roshansk/YelpAnalysis/utils.py:530: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.

200it [00:00, 244.60it/s]


...

yelp_subset , 0.385 , 0.3263964950711939 , 0.758 , 0.858


0it [00:00, ?it/s]/home/roshansk/YelpAnalysis/utils.py:530: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.

38it [00:00, 134.79it/s]


empathy , 0.489247311827957 , 0.0 , 0.505 , 0.607


0it [00:00, ?it/s]/home/roshansk/YelpAnalysis/utils.py:530: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.

421it [00:02, 206.69it/s]


nrc_joy , 0.7605700712589074 , 0.6589986468200271 , 0.724 , 0.635
